In [1]:
!pip install geopandas regionmask openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 14.6 MB/s eta 0:00:00


In [2]:
#Importação das bibliotecas
import gcsfs 
import intake
import xarray as xr
import pandas as pd
import cftime
import geopandas as gpd
import regionmask
import re
import numpy as np
import fiona
import xesmf as xe
import gsw
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [3]:
#Níveis do WOA para comparação: 
new_levels = [0.00e+00, 5.00e+00, 1.00e+01, 1.50e+01, 2.00e+01, 2.50e+01,
       3.00e+01, 3.50e+01, 4.00e+01, 4.50e+01, 5.00e+01, 5.50e+01,
       6.00e+01, 6.50e+01, 7.00e+01, 7.50e+01, 8.00e+01, 8.50e+01,
       9.00e+01, 9.50e+01, 1.00e+02, 1.25e+02, 1.50e+02, 1.75e+02,
       2.00e+02, 2.25e+02, 2.50e+02, 2.75e+02, 3.00e+02, 3.25e+02,
       3.50e+02, 3.75e+02, 4.00e+02, 4.25e+02, 4.50e+02, 4.75e+02,
       5.00e+02, 5.50e+02, 6.00e+02, 6.50e+02, 7.00e+02, 7.50e+02,
       8.00e+02, 8.50e+02, 9.00e+02, 9.50e+02, 1.00e+03, 1.05e+03,
       1.10e+03, 1.15e+03, 1.20e+03, 1.25e+03, 1.30e+03, 1.35e+03,
       1.40e+03, 1.45e+03, 1.50e+03, 1.55e+03, 1.60e+03, 1.65e+03,
       1.70e+03, 1.75e+03, 1.80e+03, 1.85e+03, 1.90e+03, 1.95e+03,
       2.00e+03, 2.10e+03, 2.20e+03, 2.30e+03, 2.40e+03, 2.50e+03,
       2.60e+03, 2.70e+03, 2.80e+03, 2.90e+03, 3.00e+03, 3.10e+03,
       3.20e+03, 3.30e+03, 3.40e+03, 3.50e+03, 3.60e+03, 3.70e+03,
       3.80e+03, 3.90e+03, 4.00e+03, 4.10e+03, 4.20e+03, 4.30e+03,
       4.40e+03, 4.50e+03, 4.60e+03, 4.70e+03, 4.80e+03, 4.90e+03,
       5.00e+03, 5.10e+03, 5.20e+03, 5.30e+03, 5.40e+03, 5.50e+03]


In [4]:
def rename_coords(ds):
    """Renomeia as variáveis de latitude, longitude e profundidade para 'lat', 'lon' e 'lev',
    respectivamente, usando os nomes de variáveis de coordenadas encontrados automaticamente no arquivo.
    """
    # Cria um dicionário com os possíveis nomes antigos das variáveis de latitude, longitude e profundidade
    # e seus respectivos novos nomes
    coord_names = {
        'latitude': 'lat', 'nav_lat': 'lat', 'lat': 'lat',
        'longitude': 'lon', 'nav_lon': 'lon', 'lon': 'lon',
        'olevel': 'lev',
        'olevel_bounds': 'lev_bnds'
    }
    # Itera sobre a lista de nomes de coordenadas presentes no arquivo
    for coord_name in ds.coords.keys():
        # Verifica se o nome da coordenada corresponde a um dos possíveis nomes antigos das variáveis de coordenadas
        if coord_name in coord_names:
            # Renomeia a variável de coordenada usando o método rename()
            ds = ds.rename({coord_name: coord_names[coord_name]})
    # Retorna o Dataset com as variáveis de coordenadas renomeadas
    return ds.copy()

#Função para acertar a variável tempo! Pois alguns formatos disponiveis no CMIP6 para a variável time dificultam sua manipulação. 
def to_360day_monthly(da):
    ''' Conversão da dimensão de tempo de modelos climáticos.
        Função criada por Claire Carouge no CLEX CMS Blog'''
    val = da.copy()
    time1 = da.time.copy()
    for itime in range(val.sizes['time']):
        bb = val.time.values[itime].timetuple()
        time1.values[itime] = cftime.Datetime360Day(bb[0],bb[1],16)
    val = val.assign_coords({'time':time1})
    return val

#Função que converte a profundidade de centímetros para metro.
def depth_m(ds):
    if "lev" in ds:
        if "units" in ds["lev"].attrs:
            units = ds["lev"].units.lower()
            if units == "cm" or units == "centimeters":
                ds["lev"] = ds["lev"] / 100
                ds["lev"].attrs["units"] = "m"
    return ds

In [68]:
# Supondo que você já tenha um DataFrame chamado cmip6.df
# Substitua o nome do DataFrame pela variável que você está utilizando

# Lista de modelos para ssp370 e thetao
modelos_ssp370 = cmip6.df.loc[(cmip6.df.experiment_id == "ssp370") & (cmip6.df.variable_id == "thetao")].source_id.unique()

# Lista de modelos para ssp585 e thetao
modelos_ssp585 = cmip6.df.loc[(cmip6.df.experiment_id == "ssp585") & (cmip6.df.variable_id == "thetao")].source_id.unique()

# Encontrar a interseção entre as duas listas
modelos_em_ambas_as_listas = set(modelos_ssp370).intersection(modelos_ssp585)

# Exibir os modelos encontrados
print("Modelos em ambas as listas:", modelos_em_ambas_as_listas)


Modelos em ambas as listas: {'CAMS-CSM1-0', 'CNRM-ESM2-1', 'MIROC-ES2L', 'MIROC6', 'EC-Earth3', 'MCM-UA-1-0', 'NorESM2-LM', 'CNRM-CM6-1-HR', 'ACCESS-CM2', 'CanESM5-CanOE', 'CNRM-CM6-1', 'GISS-E2-1-G', 'CMCC-CM2-SR5', 'MRI-ESM2-0', 'BCC-CSM2-MR', 'NorESM2-MM', 'CESM2-WACCM', 'GFDL-ESM4', 'EC-Earth3-Veg', 'INM-CM5-0', 'MPI-ESM1-2-LR', 'INM-CM4-8', 'CanESM5', 'FGOALS-g3', 'FGOALS-f3-L', 'CMCC-ESM2', 'IPSL-CM6A-LR', 'EC-Earth3-Veg-LR', 'TaiESM1', 'UKESM1-0-LL', 'CESM2', 'MPI-ESM1-2-HR', 'AWI-CM-1-1-MR', 'CAS-ESM2-0'}


In [67]:
# Lista de modelos para ssp370 e thetao
cmip6.df.loc[(cmip6.df.experiment_id == "historical") & (cmip6.df.variable_id == "so")&(cmip6.df.source_id == "ACCESS-CM2")]

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
374924,CMIP,CSIRO-ARCCSS,ACCESS-CM2,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/...,NaN,20191108
386831,CMIP,CSIRO-ARCCSS,ACCESS-CM2,historical,r2i1p1f1,Omon,so,gn,gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/...,NaN,20191125
418986,CMIP,CSIRO-ARCCSS,ACCESS-CM2,historical,r3i1p1f1,Omon,so,gn,gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/...,NaN,20200306
512321,CMIP,CSIRO-ARCCSS,ACCESS-CM2,historical,r5i1p1f1,Omon,so,gn,gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/...,NaN,20210607
512344,CMIP,CSIRO-ARCCSS,ACCESS-CM2,historical,r4i1p1f1,Omon,so,gn,gs://cmip6/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/...,NaN,20210607


In [5]:
#Acessa o conjunto de metadados do CMIP6 hospedados no Google Cloud.
cmip6 = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

In [13]:
#Lendo a minha Tabela de Modelo.
#Verifique o caminho, no meu jupyter lab está tudo na mesma pasta.
df = pd.read_excel("Tabela_Modelos/Tabela_Modelos.xlsx", sheet_name=1)
df = df.rename(columns = lambda x : x.strip())

In [7]:
gdf = gpd.read_file('Area_Projeto/Regiao_Juntas/Regiao_Juntas.shp')

In [34]:
pesquisa = {'source_id': 'GFDL-CM4',
            'table_id': 'Omon',
            'variable_id': ["thetao", "so"],
            'experiment_id': "ssp585",
            'member_id': "r1i1p1f1"}

In [ ]:
cat = cmip6.search(require_all_on='source_id', **pesquisa)
cat = cat.to_dataset_dict(aggregate=True,
                          storage_options={'token': 'anon'},
                          xarray_open_kwargs={'consolidated': True,
                                             'decode_times': True,
                                             'use_cftime': True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/srv/conda/envs/notebook/lib/python3.10/site-packages/intake_esm/_search.py:80: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in grouped:


In [14]:
#Realiza minha pesquisa de acordo com a minha Tabela de Modelos
pesquisas = []
for index, row in df.iterrows():
    pesquisa = {'source_id': row['source_id'],
                'table_id': row['table_id'],
                'variable_id': row['variable_id'].split(', '),
                'experiment_id': row['experiment_id'],
                'member_id': row['member_id']}
    pesquisas.append(pesquisa)

In [ ]:
import pandas as pd

for pesquisa in pesquisas:
    cat = cmip6.search(require_all_on='source_id', **pesquisa)
    cat = cat.to_dataset_dict(aggregate=True,
                              storage_options={'token': 'anon'},
                              xarray_open_kwargs={'consolidated': True,
                                                 'decode_times': True,
                                                 'use_cftime': True})
    
    # Variável para armazenar o dataset
    ds = None

    for key in list(cat.keys()):
        if key.endswith("gn"):
            ds = cat[key]
            break  # Se encontrou "gn", não é necessário continuar procurando

    #Faz o squeeze para conseguir fazer o regrid
    ds = ds[["thetao", "so"]].squeeze()
    
    #Transforma as coordenadas verticais para metro
    ds = depth_m(ds)
    
    
    if "gn" in ds.grid_label:
        ds_out = xr.Dataset(
            {
                "lat" : (["lat"], np.arange(-90, 90, 1.0)),
                "lon" : (["lon"], np.arange(-180, 180, 1.0)),
            }
        )

        regridder = xe.Regridder(ds, ds_out, "bilinear", ignore_degenerate=True, periodic=True)
        ds_out = regridder(ds, keep_attrs = True)
    
    else:
        ds_out = ds
    
    #Renomeia olevel para level
    ds_out = rename_coords(ds_out)
    
    #Pressão a partir da profundidade
    ds_out["press"] = gsw.p_from_z(-ds_out.lev, ds_out.lat)
    
    #Transforma salinidade preformada em salinidade absoluta
    ds_out["sa"] = gsw.SA_from_Sstar(ds_out.so, ds_out.press ,ds_out.lon, ds_out.lat)
    
    # Interpolação vertical usando interpolação linear
    ds_interp = ds_out.interp(lev=new_levels, method='linear')
    
    #Converte o GeoDataFrame para um objeto region mask
    mask = regionmask.mask_geopandas(gdf, ds_interp['lon'], ds_interp['lat'])

    #Aplica a máscara ao dataset
    ds_masked = ds_interp.where(mask==mask, drop=True)
    
    ds_lat = ds_masked.mean(dim="lon")
    
    ds_lat.to_netcdf("{}.nc".format(key))